In [ ]:
import pandas as pd  #ignorieren, ist für kontrolle | pandas hatte neue version 

print(pd.__version__)  #datenanalse& visualisierung/ manipulation in py 





2.2.3


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split #ML training & test
from sklearn.ensemble import RandomForestRegressor #ML 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import joblib

#alle pakete für datenhandling& ml training und modelle speichern 


In [ ]:
# vwerbindung zur datenbank 
#laden der datenbank
#sqlite3 
DB_PATH = Path("../data/personal.db")
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query("SELECT * FROM mitarbeiter", conn)
conn.close()

print("\nVorschau auf die Daten:")
print(df.head())



Vorschau auf die Daten:
     id  alter_jahre wohnlage  entfernung_km  homeoffice_tage  \
0  2261           49  neutral           13.4                3   
1  2262           49  neutral           46.3                7   
2  2263           41      gut            4.5                3   
3  2264           52  neutral           26.3                9   
4  2265           20  neutral           34.5                5   

  beziehungsstatus  kinder  krankheitstage  
0        verwitwet       0               7  
1      verheiratet       0               2  
2      verheiratet       1               3  
3            ledig       0               4  
4            ledig       0               5  


In [ ]:
# feature engineering und ziel 
X = df.drop(columns=["id", "krankheitstage"])  # alle spalten ausser id und k tage 
# diese sollen vorhergesagt werden(prognose), dehalb drop


y = df["krankheitstage"] # zielvariable (target variable)
# y ist die spalte die vorhergesagt werden soll ==  also die krankheitstage


In [ ]:
#One-Hot-Encoding der kategorialen Features
categorical_cols = ["wohnlage", "beziehungsstatus"] 
ohe = OneHotEncoder(sparse_output=False, drop="first")
ohe_features = ohe.fit_transform(X[categorical_cols]) 
ohe_df = pd.DataFrame(ohe_features, columns=ohe.get_feature_names_out(categorical_cols))


In [ ]:
#kombinieren mit numerischen Features
X_numeric = X.drop(columns=categorical_cols).reset_index(drop=True)
X_final = pd.concat([X_numeric, ohe_df], axis=1)


In [ ]:
#  vernwedet train_test_split um die daten in train und test zu splitten
# 80% für Training und 20% für Test
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42) # 42 ist der random state, damit die aufteilung immer gleich bleibt


In [ ]:
#  Modell training
model = RandomForestRegressor(n_estimators=100, random_state=42)     #insgesamt 100 Bäume

# 100 bäume die gemittelt werden um Vorhersage zu machen

model.fit(X_train, y_train) #modell trainieren mit  train data


RandomForestRegressor(random_state=42)

In [ ]:
#  evaluation
y_pred = model.predict(X_test) #vorhersage der testdaten


#übliche metrik für regression

#mae = mean absolute error = mittlere absolute fehler = wie weit die Vorhersage im Durchschnitt vom tatsächlichen Wert abweicht
#rmse = root mean square error = quadfurwurzel des mittl. quadr. fehler = wie weit die Vorhersage im Durchschnitt vom tatsächlichen Wert abweicht, aber stärker gewichtet für größere Fehler
#r² = Anteil der erklärten Varianz = wie gut das Modell die Daten erklärt, 1=perfekte Vorhersage, 0=keine Vorhersage

#speoicherung
print("\n Evaluation:")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}") #mittlere absolute fehler
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.2f}") #quadfurwurzel des mittl. quadr. fehler
print(f"R² Score: {r2_score(y_test, y_pred):.2f}")   #anteil der erklärten varianz



 Evaluation:
MAE: 1.87
RMSE: 2.30
R² Score: 0.54


In [ ]:
#modell speichern
MODEL_PATH = Path("../models/rf_regressor.pkl")   #pfad für das speicher des modells
joblib.dump(model, MODEL_PATH)
print(f"\n Modell gespeichert unter: {MODEL_PATH.resolve()}")



 Modell gespeichert unter: C:\projects\project-decision-agent\decision-agent\models\rf_regressor.pkl
